In [ ]:
import numpy as np #Import Librarys
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pandas_profiling
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
import warnings
import plotly.express as px
from sklearn import metrics
from sklearn.preprocessing import Normalizer
import xgboost as xgb
from sklearn.model_selection import cross_val_score
import lightgbm as lgb
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from collections import Counter
import pickle #for save Model
from sklearn.ensemble import RandomForestClassifier
#2 sub-sets: personal information and transactional records

In [ ]:
df_app = pd.read_csv("/application_record.csv") #Load CSV

In [ ]:
df_app.info()

In [ ]:
df_app.head()

In [ ]:
df_app.describe()

In [ ]:
df_app['FLAG_OWN_REALTY'].unique() 

In [ ]:
df_app['NAME_INCOME_TYPE'].unique() 

In [ ]:
df_app['OCCUPATION_TYPE'].unique() 

In [ ]:
df_rec = pd.read_csv("/credit_record.csv")


In [ ]:
df_rec.head()

In [ ]:
df_rec.info()

In [ ]:
df_rec['STATUS'].unique() 

In [ ]:
#X signifie aucun pret de mois
#C signifie qu'il est rembosé 
df_rec.STATUS = df_rec.STATUS.replace ({'X': - 2, 'C': -1}) 

In [ ]:
df_rec.isnull().values.any()

In [ ]:
df_rec.isnull().sum().sum()

In [ ]:
df_rec=df_rec.dropna()

In [ ]:
df_rec.info()

In [ ]:
df_rec.STATUS = df_rec.STATUS.astype ('int')
df_rec.STATUS = df_rec.STATUS.apply ( lambda x: x + 1)

In [ ]:
duplicateRowsDF = df_rec[df_rec.duplicated()]
print("Duplicate Rows except first occurrence based on all columns are :")
print(duplicateRowsDF)

In [ ]:
credit_month = df_rec.groupby ('ID'). MONTHS_BALANCE.max ().reset_index()

In [ ]:
record = pd.merge (credit_month, df_rec, how = "inner", on = ["ID", "MONTHS_BALANCE"])

In [ ]:
len(set(record['ID']))

In [ ]:
dataRisk = pd.merge(df_app, record, how ='inner', on = ['ID'])

In [ ]:
len(set(dataRisk['ID']))

In [ ]:
dataRisk.info()

In [ ]:
dataRisk['AGE']=dataRisk['DAYS_BIRTH'].apply(lambda x : round(x/-365.0))

In [ ]:
dataRisk.drop('DAYS_BIRTH', axis='columns', inplace=True)

In [ ]:
dataRisk['DAYS_EMPLOYED'] = dataRisk.DAYS_EMPLOYED.apply(lambda x: round (x / -365,0) if x<0 else 0)

In [ ]:
dataRisk.isnull (). sum ()

In [ ]:
dataRisk.OCCUPATION_TYPE = dataRisk.OCCUPATION_TYPE.fillna("Others")

In [ ]:
dataRisk.describe()

In [ ]:
indices = dataRisk[dataRisk['DAYS_EMPLOYED']>=20].index
dataRisk.drop(indices, inplace=True)

In [ ]:
boxplot = dataRisk.boxplot(column=['CNT_FAM_MEMBERS'])

In [ ]:
indices = dataRisk[dataRisk['CNT_FAM_MEMBERS']>=5].index
dataRisk.drop(indices, inplace=True)

In [ ]:
boxplot = dataRisk.boxplot(column=['CNT_FAM_MEMBERS'])

In [ ]:
boxplot = dataRisk.boxplot(column=['AMT_INCOME_TOTAL'])

In [ ]:
indices = dataRisk[dataRisk['AMT_INCOME_TOTAL']>=390000].index
dataRisk.drop(indices, inplace=True)

In [ ]:
boxplot = dataRisk.boxplot(column=['CNT_CHILDREN'])

In [ ]:
indices = dataRisk[dataRisk['CNT_CHILDREN']>=3].index
dataRisk.drop(indices, inplace=True)

In [ ]:
boxplot = dataRisk.boxplot(column=['CNT_CHILDREN'])

In [ ]:
cols = dataRisk.columns.tolist()

In [ ]:
cols = cols[:2]+cols[-1:]+cols[2:-1]

In [ ]:
cols

In [ ]:
dataRisk=dataRisk[cols]

In [ ]:
dataRisk.info()

In [ ]:

dataRisk.loc[dataRisk.STATUS < 1,'Risk']=0
dataRisk.loc[dataRisk.STATUS >= 1,'Risk']=1


In [ ]:
dataRisk.drop(['STATUS'], axis='columns', inplace=True)

In [ ]:
dataRisk.info()

In [ ]:
dataRisk.drop(['ID','FLAG_OWN_REALTY'], axis='columns', inplace=True)

In [ ]:
dataRisk.info()

In [ ]:
#Rename columns
dataRisk = dataRisk.rename(columns={'CODE_GENDER':'Gender',
                   'FLAG_OWN_CAR':'car',
                   'AMT_INCOME_TOTAL':'annual_income',
                   #'NAME_INCOME_TYPE':'income_type',
                   'NAME_EDUCATION_TYPE':'education',
                   'NAME_FAMILY_STATUS':'family_status',
                   'NAME_HOUSING_TYPE':'house',
                   'AGE':'age',
                   'DAYS_EMPLOYED':'employment',
                   'CNT_FAM_MEMBERS':'fam_members',
                   'OCCUPATION_TYPE': 'occupation'
                   })
dataRisk.head()

In [ ]:
dataRisk.info()

In [ ]:
dataRisk.drop(['CNT_CHILDREN','FLAG_MOBIL','FLAG_WORK_PHONE','FLAG_PHONE','MONTHS_BALANCE','NAME_INCOME_TYPE','education'], axis='columns', inplace=True)

In [ ]:
dataRisk.drop(['FLAG_EMAIL'], axis='columns', inplace=True) #'ID','FLAG_OWN_REALTY',

In [ ]:
dataRisk.head(4)

In [ ]:
sns.set(font_scale=1,style="whitegrid")
fig,ax=plt.subplots(ncols=2,nrows=2,figsize=(16,12))
cat_list=["age","annual_income"]
count=0
for i in range(2):
    sns.distplot(dataRisk[cat_list[count]],ax=ax[i][0],kde=False,color="#F43EEC")
    sns.kdeplot(dataRisk[cat_list[count]],ax=ax[i][1],shade=True,color="#359F4B")
    count+=1

In [ ]:
dataRisk[dataRisk['age']>65]

In [ ]:
display(dataRisk['Gender'].value_counts())


In [ ]:
dataRisk['Gender'].replace('F',0,inplace=True)
dataRisk['Gender'].replace('M',1,inplace=True)  

In [ ]:
dataRisk.Gender = dataRisk.Gender.astype ('int')

In [ ]:
display(dataRisk['family_status'].value_counts())



In [ ]:
dataRisk['family_status'] = dataRisk['family_status'].apply(lambda x: 1 if ((x == 'Married') or (x == 'Civil marriage')) else 0)

In [ ]:
dataRisk.family_status= dataRisk.family_status.astype ('int')

In [ ]:
display(dataRisk['house'].value_counts())


In [ ]:
dataRisk['house'] = dataRisk['house'].apply(lambda x: 1 if x == 'House / apartment' else 0)

In [ ]:
dataRisk.house= dataRisk.house.astype ('int')

In [ ]:
display(dataRisk['occupation'].value_counts())

In [ ]:
display(dataRisk['car'].value_counts())

In [ ]:
dataRisk['car'].replace('N',0,inplace=True)
dataRisk['car'].replace('Y',1,inplace=True)

In [ ]:
dataRisk.car= dataRisk.car.astype ('int')


In [ ]:

#Detect and remove outliers: Annual_income

plt.figure(figsize=(13,7))
sns.boxplot(dataRisk.annual_income)

In [ ]:
indices = dataRisk[dataRisk['annual_income']>=390000].index
dataRisk.drop(indices, inplace=True)

In [ ]:
plt.figure(figsize=(13,7))
sns.boxplot(dataRisk.annual_income)

In [ ]:
px.box(data_frame=dataRisk, x='occupation', y='annual_income')

In [ ]:
dataRisk.drop(dataRisk.query('occupation=="Security staff" and annual_income > 350000').index, inplace=True)
dataRisk.drop(dataRisk.query('occupation=="Sales staff" and annual_income > 370000').index, inplace=True)
dataRisk.drop(dataRisk.query('occupation=="Others" and annual_income > 340000').index, inplace=True)
dataRisk.drop(dataRisk.query('occupation=="Drivers" and annual_income > 350000').index, inplace=True)
dataRisk.drop(dataRisk.query('occupation=="Medicine staff" and annual_income > 350000').index, inplace=True)
dataRisk.drop(dataRisk.query('occupation=="Secretaires" and annual_income > 350000').index, inplace=True)
dataRisk.drop(dataRisk.query('occupation=="Waiters/barmen staff" and 80000 > annual_income > 200000').index, inplace=True)
dataRisk.drop(dataRisk.query('occupation=="HR staff" and annual_income > 250000').index, inplace=True)
dataRisk.drop(dataRisk.query('occupation=="Low-skill Laborers" and annual_income > 250000').index, inplace=True)
dataRisk.drop(dataRisk.query('occupation=="Realty agents" and 150000 > annual_income >350000').index, inplace=True)
dataRisk.drop(dataRisk.query('occupation=="IT staff" and annual_income > 320000').index, inplace=True)
dataRisk.drop(dataRisk.query('occupation=="Cooking staff" and annual_income > 250000').index, inplace=True)
dataRisk.drop(dataRisk.query('occupation=="Private service staff" and annual_income > 370000').index, inplace=True)

In [ ]:
dataRisk.drop(dataRisk.query('occupation=="Sales staff" and annual_income > 350000').index, inplace=True)
dataRisk.drop(dataRisk.query('occupation=="Accountants" and annual_income > 360000').index, inplace=True)
dataRisk.drop(dataRisk.query('occupation=="Laborers" and annual_income > 360000').index, inplace=True)
dataRisk.drop(dataRisk.query('occupation=="Core staff" and annual_income > 370000').index, inplace=True)
dataRisk.drop(dataRisk.query('occupation=="Cooking staff" and annual_income > 230000').index, inplace=True)
dataRisk.drop(dataRisk.query('occupation=="Low-skill Laborers" and annual_income > 200000').index, inplace=True)
dataRisk.drop(dataRisk.query('occupation=="Waiters/barmen staff" and 75000 > annual_income > 200000').index, inplace=True)
dataRisk.drop(dataRisk.query('occupation=="Secretaires" and annual_income > 337500').index, inplace=True)
dataRisk.drop(dataRisk.query('occupation=="Realty agents" and 150000> annual_income').index, inplace=True)

In [ ]:
px.box(data_frame=dataRisk, x='occupation', y='annual_income')

In [ ]:
dataRisk.drop(dataRisk.query('occupation=="Waiters/barmen staff" and 76500 >= annual_income >= 225000').index, inplace=True)
dataRisk.drop(dataRisk.query('occupation=="Secretaires" and annual_income >= 337500').index, inplace=True)


In [ ]:
px.box(data_frame=dataRisk, x='occupation', y='annual_income')

In [ ]:
OCCUPATION_TYPE = pd.get_dummies(dataRisk['occupation'], drop_first=True)

In [ ]:
%matplotlib inline
pd.crosstab(dataRisk.annual_income,dataRisk.occupation).plot(kind='bar',figsize=(15,10))
#plt.title('')        
plt.xlabel('annual income')
plt.ylabel('occupation')
plt.savefig('occupation && annual income')



In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(x=dataRisk['occupation'], y=dataRisk['annual_income'], palette="Reds_r")
plt.xlabel('\nOccupation', fontsize=15, color='#c0392b')
plt.ylabel("Annual Income\n", fontsize=15, color='#c0392b')
plt.title("Annual Income && Occupation \n", fontsize=18, color='#e74c3c')
plt.xticks(rotation= 45)
plt.tight_layout()

In [ ]:
df=dataRisk[['occupation', 'annual_income']]
df2=df.groupby(['occupation']).mean()
df2=df2.sort_values('annual_income',ascending=False)
df2


In [ ]:
#,'Accountants'
OCCUPATION_TYPE['CLASS 3 WORKERS'] = OCCUPATION_TYPE[['Low-skill Laborers','Cooking staff','Cleaning staff']].sum(axis=1)
OCCUPATION_TYPE['CLASS 2 WORKERS'] = OCCUPATION_TYPE[['Medicine staff','HR staff','Waiters/barmen staff','Security staff','Sales staff','Secretaries','IT staff']].sum(axis=1)
OCCUPATION_TYPE['CLASS 1 WORKERS'] = OCCUPATION_TYPE[['Laborers','Core staff','Private service staff','High skill tech staff','Drivers','Managers','Realty agents']].sum(axis=1)


In [ ]:
OCCUPATION_TYPE
OCCUPATION_TYPE.drop(['Cleaning staff','Cooking staff','Core staff','Drivers','HR staff','High skill tech staff','IT staff','Laborers','Low-skill Laborers','Managers','Medicine staff','Private service staff','Realty agents','Sales staff','Secretaries','Security staff','Waiters/barmen staff'], axis=1,inplace=True)

In [ ]:
dataRisk.drop(['occupation'], axis='columns', inplace=True)

In [ ]:
dataRisk = pd.concat([dataRisk, OCCUPATION_TYPE], axis=1)

In [ ]:
dataRisk.head(4)

In [ ]:
dataRisk.dtypes

In [ ]:
dataRisk.info()

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
sns.heatmap(dataRisk.corr(), annot=True, cmap='Greens',ax=ax)
plt.title("Matrice de corrélation entre les différentes caractéristiques des clients\n", fontsize=18, color='#009432')
import matplotlib.pyplot as plt


In [ ]:
# Continuous variables: annual_income, age, year_of_employment 

fig, ax = plt.subplots(2,2,figsize=(15,10))

ax[0][0].hist(dataRisk.age)
ax[0][1].hist(dataRisk.annual_income)
ax[1][0].hist(dataRisk.employment)
ax[1][1].hist(dataRisk.fam_members) # fam_members

In [ ]:
X = dataRisk.drop('Risk', axis=1)
Y = dataRisk[['Risk']]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [ ]:
plt.hist(X_train['annual_income'], label='X_train');
plt.hist(X_test['annual_income'], label='X_test');
plt.legend();
plt.title('Distribution of Annual_income');

In [ ]:
risp=dataRisk[dataRisk['Risk']==1.0]

In [ ]:
risn=dataRisk[dataRisk['Risk']==0.0]

In [ ]:
risp.info()

In [ ]:
risn.info()

In [ ]:
# nombre de classes
class_count_0, class_count_1 = dataRisk['Risk']. value_counts ()

# Classe séparée
class_0 = dataRisk[dataRisk['Risk'] == 0]
class_1 = dataRisk[dataRisk['Risk'] == 1] # imprime la forme de la classe
print ('classe 0:', class_0.shape)
print ('classe 1:', class_1.shape)

In [ ]:
#Dealing with imbalanced data (Oversampling (SMOTE))


class_00 = dataRisk[dataRisk.Risk == 0].Risk.count() / dataRisk.Risk.count()
class_11 = dataRisk[dataRisk.Risk == 1].Risk.count() / dataRisk.Risk.count()

print(class_00)
print(class_11)

In [ ]:
duplicateRowsDF = dataRisk[dataRisk.duplicated()]
print("Duplicate Rows except first occurrence based on all columns are :")
print(duplicateRowsDF)


In [ ]:
dataRisk=dataRisk.drop_duplicates() #Remove duplicate rows

In [ ]:
dataRisk.info()

In [ ]:
%matplotlib inline
pd.crosstab(dataRisk.age,dataRisk.annual_income).plot(kind='bar',figsize=(15,10))
#plt.title('')        
plt.xlabel('age')
plt.ylabel('annual income')
plt.savefig('age && annual income')



In [ ]:
#Oversampling (SMOTE: Synthetic Minority Oversampling Technique)

from imblearn.over_sampling import SMOTE
from collections import Counter

oversample = SMOTE()

x_train_over, y_train_over = oversample.fit_resample(X_train, Y_train)
print(Counter(y_train_over))

In [ ]:
X_train.columns

In [ ]:
#Apply to x_test_set
X_test_over, Y_test_over = oversample.fit_resample(X_test, Y_test)
print(Counter(Y_test_over))


In [ ]:
#lightGBM
from sklearn.metrics import accuracy_score
import pickle
import lightgbm as lgb
lgb_params = {
                'objective':'binary',
                'boosting_type':'gbdt',
                 'metric':'auc',
                'n_jobs':-1,
                'learning_rate':0.1,
                'max_depth':12,
                'n_estimators':1000,
                'seed': 0,
                'early_stopping_rounds':100, 
            }
clf = lgb.LGBMClassifier(**lgb_params)
clf.fit(
        x_train_over, y_train_over,
        eval_set=[(x_train_over, y_train_over),(X_test_over, Y_test_over)],
        verbose=50,
        early_stopping_rounds=100)
ypred1 = clf.predict_proba(X_test,num_iteration=clf.best_iteration_)[:,1]
# predict the results


val_score = roc_auc_score(Y_test, ypred1)
print(f"val_score = {val_score}")


Pkl_Filename = "Risk_modelGBM.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(clf, file)
loaded_model = pickle.load(open(Pkl_Filename, 'rb'))
result = loaded_model.score(X_test, Y_test)
print("XGBMLIGHT model created")
  

In [ ]:
y_pred_test=clf.predict(X_test)
acc1=clf.score(x_train_over, y_train_over)
acc2=clf.score(X_test, Y_test)
print('training accuarcy  is:{0:0.4f}'.format(acc1))
print('test accuarcy is:{0:0.4f}'.format(acc2))
val_score = roc_auc_score(Y_test, ypred1)
print("auc score lightGBM model is:{0:0.4f}".format(val_score))
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(Y_test,y_pred_test)))

In [ ]:
print(classification_report(Y_test, y_pred_test))